In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from IPython.display import clear_output
from data_scaler import DataScaler
import matplotlib.pyplot as plt
import numpy as np 
from pprint import pprint

from model_trainer import ModelTrainer
from data_loader import DataLoader
from physics_engine import generate_data_with_simulator, N_FEATURES, OUTPUT

### N body
In the n-body domain,  such as solar systems, all n bodies exert distance- and mass-dependent gravitational forces on each other, so there were n(n−1) relations input to our model.  Across simulations, the objects’ masses varied, while all other fixed attributes were held constant

In [3]:
n_samples = 10
n_objects  = 6
relation_dim = 1
effect_dim = 100  # effect's vector size
dt = 0.001

batch_size = 1000
time_steps = 100000
n_epoch = 100

In [15]:
n_objects  = 6
relation_dim = 1
effect_dim = 100  # effect's vector size
dt = 0.001

batch_size = 100
n_samples = 4
time_steps = 10000
n_epoch = 100

In [4]:
data = generate_data_with_simulator(n_samples=n_samples, n_objects=n_objects, time_steps=time_steps, dt=dt)
scaler = DataScaler(data)

data_loader = DataLoader(original_data=data,
                         scaler=scaler,
                         batch_size=batch_size,
                         shuffle=True)

print('len data', len(data))
print('data loader len', len(data_loader))

In [12]:
model = ModelTrainer(state_dim=N_FEATURES,
                     relation_dim=relation_dim,
                     effect_dim=effect_dim,
                     output_dim=len(OUTPUT))

In [17]:
model.train(n_epoch=n_epoch,
            data_loader=data_loader)

In [9]:
cycles = 5
for ii in range(cycles):
    print(f'Cycle {ii+1}/{cycles}')
    data = generate_data_with_simulator(n_samples=n_samples, n_objects=n_objects, time_steps=time_steps, dt=dt)
    data_loader = DataLoader(original_data=data, scaler=scaler, batch_size=batch_size, shuffle=True)    
    model.train(n_epoch=n_epoch, data_loader=data_loader)

print('Finished')

In [41]:
model.save(time_stamped=True)

In [5]:
model = ModelTrainer.load()
scaler = model.scaler

In [40]:
model.losses = np.array(model.losses, dtype=np.float32)
print('losses', len(model.losses), type(model.losses), type(model.losses[0]), len(model.losses)*32/8/2**20)
plt.figure(figsize=(10, 5))
# plt.plot(model.losses[-int(len(model.losses)/2):])
plt.plot(model.losses[1000:])
plt.show()


In [43]:
np.array(range(10)).tolist()

In [31]:
n_objects_test = 4
test_data = generate_data_with_simulator(n_samples=1,
                                         n_objects=n_objects_test, 
                                        orbit=not True,
                                        time_steps=200, 
                                        dt=0.001)
test_data_loader = DataLoader(original_data=test_data, 
                              scaler=scaler,
                              batch_size=None,
                              shuffle=False)

sim_output = model.test(data_loader=test_data_loader, dt=0.001)